<a href="https://colab.research.google.com/github/Vishakhajoshi24/Fynd-AI-Intern-Assessment/blob/main/task1_fynd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install -q openai pandas tqdm

In [45]:
import pandas as pd
import json
import time
from tqdm import tqdm
from openai import OpenAI
import os

In [46]:
os.environ["OPENROUTER_API_KEY"] = "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"

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"]
)

In [47]:
resp = client.chat.completions.create(
    model="mistralai/mistral-7b-instruct",
    messages=[{"role": "user", "content": "Reply with only the word OK"}],
    temperature=0
)

print(resp.choices[0].message.content)

 OK


In [50]:
df = pd.read_csv("/content/yelp.csv.zip")  # change if filename differs
df = df[['text', 'stars']]
df.head()

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [51]:
df_sample = df.sample(n=200, random_state=42).reset_index(drop=True)

In [52]:
def call_llm(prompt):
    try:
        resp = client.chat.completions.create(
            model="mistralai/mistral-7b-instruct",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        text = resp.choices[0].message.content.strip()
        text = text.replace("```json", "").replace("```", "")
        start = text.find("{")
        end = text.rfind("}") + 1
        return json.loads(text[start:end])
    except:
        return None

In [53]:
def prompt_v1(review):
    return f"""
Predict the star rating (1 to 5) for the following Yelp review.

Review:
"{review}"

Return JSON:
{{
  "predicted_stars": number,
  "explanation": "short reason"
}}
"""

In [54]:
results_v1 = []

for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    results_v1.append(call_llm(prompt_v1(row['text'])))
    time.sleep(0.2)

df_sample['pred_v1'] = results_v1

100%|██████████| 200/200 [03:08<00:00,  1.06it/s]


In [56]:
def evaluate(df, col):
    valid = df[col].notnull()

    if valid.sum() == 0:
        return {"accuracy": 0.0, "json_validity": 0.0}

    accuracy = sum(
        df.loc[valid, col].apply(lambda x: x['predicted_stars'])
        == df.loc[valid, 'stars']
    ) / valid.sum()

    return {
        "accuracy": round(accuracy * 100, 2),
        "json_validity": round(valid.mean() * 100, 2)
    }


In [57]:
evaluate(df_sample, 'pred_v1')

{'accuracy': np.float64(54.55), 'json_validity': np.float64(27.5)}

PROMPT 2

In [59]:
def prompt_v2(review):
    return f"""
You are an expert Yelp sentiment analyst.

Classify the review into a star rating from 1 to 5.

Rules:
- Return ONLY valid JSON
- predicted_stars must be integer between 1 and 5
- explanation max 1 sentence

Review:
"{review}"

JSON format:
{{
  "predicted_stars": 3,
  "explanation": "Reason"
}}
"""

In [60]:
results_v2 = []

for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    results_v2.append(call_llm(prompt_v2(row['text'])))
    time.sleep(0.2)

df_sample['pred_v2'] = results_v2

100%|██████████| 200/200 [05:02<00:00,  1.51s/it]


In [61]:
evaluate(df_sample, 'pred_v2')

{'accuracy': np.float64(57.23), 'json_validity': np.float64(79.5)}

PROMPT 3

In [62]:
def prompt_v3(review):
    return f"""
Examples:

Review: "Amazing food and friendly staff."
Output: {{"predicted_stars": 5, "explanation": "Very positive experience."}}

Review: "Terrible service and rude staff."
Output: {{"predicted_stars": 1, "explanation": "Very negative experience."}}

Now classify:
Review: "{review}"

Return ONLY valid JSON.
"""

In [63]:
results_v3 = []

for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    results_v3.append(call_llm(prompt_v3(row['text'])))
    time.sleep(0.2)

df_sample['pred_v3'] = results_v3

100%|██████████| 200/200 [05:20<00:00,  1.60s/it]


In [64]:
evaluate(df_sample, 'pred_v3')

{'accuracy': np.float64(64.89), 'json_validity': np.float64(94.0)}

In [65]:
summary = pd.DataFrame([
    {"Prompt": "Basic", **evaluate(df_sample, 'pred_v1')},
    {"Prompt": "Role-Based", **evaluate(df_sample, 'pred_v2')},
    {"Prompt": "Few-Shot", **evaluate(df_sample, 'pred_v3')}
])

summary

,Prompt,accuracy,json_validity
0,Basic,54.55,27.5
1,Role-Based,57.23,79.5
2,Few-Shot,64.89,94.0
